In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta
import joblib

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# =============================================
# Step 1. Load all merged Parquet files
# =============================================
def load_all_merged_data(meteo_dir: Path) -> dict:
    """
    Loads all Parquet files from the given directory into a dictionary,
    using the file stem (plant name) as the key.
    Assumes each file contains a UTC-aware DatetimeIndex and a 'generation' column,
    along with meteorological variables.
    """
    plant_dfs = {}
    for file in meteo_dir.glob("*.parquet"):
        df = pd.read_parquet(file)
        # Ensure the index is a UTC-aware DatetimeIndex
        if not isinstance(df.index, pd.DatetimeIndex):
            df["date"] = pd.to_datetime(df["date"], utc=True)
            df = df.set_index("date")
        plant_name = file.stem  # e.g. "parque_eolico_agua_clara"
        plant_dfs[plant_name] = df
    return plant_dfs

# =============================================
# Step 2. Preprocess each DataFrame
#   a. Drop only the initial consecutive days with zero generation.
#   b. Drop the last 26 hours.
# =============================================
def drop_initial_zero_generation_days(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops the very first consecutive days where the total generation is zero.
    Once a day with non-zero generation is encountered, subsequent days are retained.
    """
    df = df.copy()
    df["date_only"] = df.index.normalize()
    unique_dates = sorted(df["date_only"].unique())
    drop_dates = []
    for date in unique_dates:
        daily_sum = df.loc[df["date_only"] == date, "generation"].sum()
        if daily_sum == 0:
            drop_dates.append(date)
        else:
            break  # Stop after the first day with non-zero generation
    df_clean = df[~df["date_only"].isin(drop_dates)].copy()
    df_clean.drop(columns=["date_only"], inplace=True)
    return df_clean

def drop_last_26_hours(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops rows from the DataFrame where the timestamp is within the last 26 hours.
    """
    last_timestamp = df.index.max()
    cutoff = last_timestamp - timedelta(hours=26)
    return df[df.index <= cutoff]

# =============================================
# Step 3. Grid Search Hyperparameter Tuning for Models
# =============================================
def tune_model_grid(model, param_grid, X_train, y_train, cv=3):
    """
    Performs grid search using GridSearchCV on the given model.
    Returns the best estimator.
    """
    grid_cv = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv,
        scoring="r2",
        n_jobs=-1
    )
    grid_cv.fit(X_train, y_train)
    return grid_cv.best_estimator_

def tune_and_evaluate_models(df: pd.DataFrame, plant: str) -> (dict, dict):
    """
    Given a preprocessed DataFrame for a plant, this function:
      - Drops rows with missing values.
      - Splits data into training (first 80%) and testing (last 20%) sets (time-ordered).
      - Uses all numeric meteorological variables (excluding 'generation') as features.
      - Trains three models:
            * Linear Regression (no tuning)
            * Random Forest (with GridSearchCV)
            * Gradient Boosting (with GridSearchCV)
      - Evaluates each model using Mean Squared Error and R².
    
    Returns two dictionaries:
      - performance: mapping model names to metrics.
      - model_objs: mapping model names to the trained model objects.
    """
    df = df.dropna()
    if df.empty:
        print(f"No data available for {plant} after dropping missing values.")
        return {}, {}
    
    X = df.drop(columns=["generation"])
    y = df["generation"]
    X = X.select_dtypes(include=[np.number])
    
    split_index = int(len(df) * 0.8)
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]
    
    performance = {}
    model_objs = {}
    
    # Linear Regression (no tuning)
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    mse_lr = mean_squared_error(y_test, y_pred_lr)
    r2_lr = r2_score(y_test, y_pred_lr)
    performance["LinearRegression"] = {"MSE": mse_lr, "R2": r2_lr}
    model_objs["LinearRegression"] = lr
    print(f"{plant} - LinearRegression: MSE = {mse_lr:.2f}, R2 = {r2_lr:.2f}")
    
    # Random Forest with GridSearchCV
    rf_param_grid = {
        "n_estimators": [50, 100],
        "max_depth": [5, 10],
        "min_samples_split": [2, 4],
        "min_samples_leaf": [1, 2]
    }
    rf = RandomForestRegressor(random_state=42)
    rf_best = tune_model_grid(rf, rf_param_grid, X_train, y_train, cv=3)
    y_pred_rf = rf_best.predict(X_test)
    mse_rf = mean_squared_error(y_test, y_pred_rf)
    r2_rf = r2_score(y_test, y_pred_rf)
    performance["RandomForest"] = {"MSE": mse_rf, "R2": r2_rf}
    model_objs["RandomForest"] = rf_best
    print(f"{plant} - RandomForest (tuned): MSE = {mse_rf:.2f}, R2 = {r2_rf:.2f}")
    
    # Gradient Boosting with GridSearchCV
    gb_param_grid = {
        "n_estimators": [50, 100],
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 5],
        "min_samples_split": [2, 4],
        "min_samples_leaf": [1, 2]
    }
    gb = GradientBoostingRegressor(random_state=42)
    gb_best = tune_model_grid(gb, gb_param_grid, X_train, y_train, cv=3)
    y_pred_gb = gb_best.predict(X_test)
    mse_gb = mean_squared_error(y_test, y_pred_gb)
    r2_gb = r2_score(y_test, y_pred_gb)
    performance["GradientBoosting"] = {"MSE": mse_gb, "R2": r2_gb}
    model_objs["GradientBoosting"] = gb_best
    print(f"{plant} - GradientBoosting (tuned): MSE = {mse_gb:.2f}, R2 = {r2_gb:.2f}")
    
    return performance, model_objs

def select_and_save_best_model(performance: dict, model_objs: dict, plant: str, model_dir: Path):
    """
    Selects the best model (highest R²) and saves it to disk using joblib.
    The model is saved with a .h5 extension.
    """
    if not performance:
        print(f"No performance metrics for {plant}. Skipping model saving.")
        return None
    best_model_name = max(performance, key=lambda m: performance[m]["R2"])
    best_model = model_objs[best_model_name]
    model_file = model_dir / f"{plant}_best_model.h5"
    joblib.dump(best_model, model_file)
    print(f"Best model for {plant} is {best_model_name} (R2 = {performance[best_model_name]['R2']:.2f}) and has been saved to {model_file}")
    return best_model_name

# =============================================
# Step 4. Main processing: Load, preprocess, model, and save results
# =============================================
def main():
    # Define paths (adjust as needed)
    meteo_dir = Path("../data/interim/meteo_data_with_generation")
    output_data_dir = Path("../data/interim/meteo_data_with_generation/processed_models")
    output_data_dir.mkdir(parents=True, exist_ok=True)
    model_dir = output_data_dir / "models"
    model_dir.mkdir(parents=True, exist_ok=True)
    
    # Load all merged data from Parquet files
    plant_dfs = load_all_merged_data(meteo_dir)
    
    overall_results = {}
    best_models = {}
    
    for plant, df in plant_dfs.items():
        print(f"\nProcessing plant: {plant}")
        # Step 2a: Drop the initial consecutive days with zero generation
        df_clean = drop_initial_zero_generation_days(df)
        # Step 2b: Drop the last 26 hours (incomplete data)
        df_clean = drop_last_26_hours(df_clean)
        # Save cleaned DataFrame (optional)
        clean_path = output_data_dir / f"{plant}_clean.parquet"
        df_clean.to_parquet(clean_path, index=True)
        print(f"Saved cleaned data for {plant} to {clean_path}")
        
        # Step 3: Train and evaluate models using meteorological features to predict generation
        if not df_clean.empty:
            performance, model_objs = tune_and_evaluate_models(df_clean, plant)
            overall_results[plant] = performance
            best_model_name = select_and_save_best_model(performance, model_objs, plant, model_dir)
            best_models[plant] = best_model_name
        else:
            print(f"No data remains for {plant} after cleaning.")
    
    # Save overall performance metrics to a CSV file for later review
    results_df = pd.DataFrame.from_dict({plant: metrics for plant, metrics in overall_results.items()}, orient="index")
    results_csv = output_data_dir / "model_performance_results.csv"
    results_df.to_csv(results_csv)
    print("\nOverall model performance results:")
    print(results_df)
    print(f"Saved performance metrics to {results_csv}")

# Execute the main function (suitable for a Jupyter Notebook)
main()


Processing plant: parque_solar_girasol
Saved cleaned data for parque_solar_girasol to ..\data\interim\meteo_data_with_generation\processed_models\parque_solar_girasol_clean.parquet
parque_solar_girasol - LinearRegression: MSE = 117.56, R2 = 0.88
parque_solar_girasol - RandomForest (tuned): MSE = 99.95, R2 = 0.90
parque_solar_girasol - GradientBoosting (tuned): MSE = 98.48, R2 = 0.90
Best model for parque_solar_girasol is GradientBoosting (R2 = 0.90) and has been saved to ..\data\interim\meteo_data_with_generation\processed_models\models\parque_solar_girasol_best_model.h5

Overall model performance results:
                                                       LinearRegression  \
parque_solar_girasol  {'MSE': 117.56472067441558, 'R2': 0.8802702746...   

                                                           RandomForest  \
parque_solar_girasol  {'MSE': 99.95428011029138, 'R2': 0.89820501905...   

                                                       GradientBoosting  
parque_so